# ТИПИС 4 дз

Здесь буду впервые пытаться решить задачу классификации с горем и болью

Классификация - это тип задачи обучения с учителем, где целью является присвоение входным данным метки из заранее определённого набора категорий (классов)

**В этом датасете целевой переменной для задачи классификации будет переменная y — клиент подписал срочный депозит или нет.**

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif


In [4]:
df = pd.read_csv('data/bank-full.csv', sep=";")

In [5]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


## Подготовка данных

**Выберите только указанные выше признаки.
Проверьте, есть ли пропущенные значения в этих признаках.**

In [6]:
# в задании требуется удалить столбцы default и loan
df = df.drop(['default', 'loan'], axis=1)
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no


In [7]:
# Проверка пропущенных значений
df.isnull().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

## Вопрос 1

**Какое самое частое значение (mode) для столбца education?**

In [8]:
df['education'].mode()

0    secondary
Name: education, dtype: object

## Вопрос 2

Создайте корреляционную матрицу для числовых признаков вашего датасета.

В корреляционной матрице вычислите коэффициент корреляции между каждой парой признаков.

**Какие два признака имеют наибольшую корреляцию?**

$$
\sigma = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})^2}
$$
$$
r_{XY} = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}
{\sqrt{\sum_{i=1}^{n} (x_i - \bar{x})^2 \cdot \sum_{i=1}^{n} (y_i - \bar{y})^2}}
$$

In [9]:
corr_matrix = df[['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']].corr()
corr_matrix

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.097783,-0.009120,-0.004648,0.004760,-0.023758,0.001288
balance,0.097783,1.000000,0.004503,0.021560,-0.014578,0.003435,0.016674
day,-0.009120,0.004503,1.000000,-0.030206,0.162490,-0.093044,-0.051710
duration,-0.004648,0.021560,-0.030206,1.000000,-0.084570,-0.001565,0.001203
campaign,0.004760,-0.014578,0.162490,-0.084570,1.000000,-0.088628,-0.032855
pdays,-0.023758,0.003435,-0.093044,-0.001565,-0.088628,1.000000,0.454820
previous,0.001288,0.016674,-0.051710,0.001203,-0.032855,0.454820,1.000000


## Кодирование целевой переменной

Теперь нам нужно закодировать переменную y.
Замените значения yes/no на 1/0.


In [10]:
# Создадим колонку target
# Такой способ придумал не сам, в курсике его взял

"""
df.loc[(df['y'] == 'no'), 'target'] = 0
df.loc[(df['y'] == 'yes'), 'target'] = 1
df = df.drop('y', axis=1)
df.head()
"""

# Чат ГПТ (вроде лучше)
df['target'] = df['y'].map({'no': 0, 'yes': 1}).astype(int)
df = df.drop('y', axis=1)
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,target
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,0


## Разделение данных

Разделите данные на **тренировочный/валидационный/тестовый** наборы с распределением 60%/20%/20%.

Используйте для этого Scikit-Learn (функцию train_test_split) и установите seed на 42.

Убедитесь, что целевая переменная y не включена в датафрейм.


In [11]:
# 1. разделяю на признаки и таргет, чуть шарю

X = df.drop('target', axis=1)
Y = df['target']

Вот дальше я вообще хз что я за магию делаю

In [12]:
# 2. делю на тренировочную и временную выборки (60/40)

X_train, X_temp, Y_train, Y_temp = train_test_split(
    X,
    Y,
    test_size=0.4,      # 40% уходит во временный набор
    random_state=42
)

In [13]:
# 3. делю временную выборку на валидационную и тестовую (по 20% каждая)
X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp,
    Y_temp,
    test_size=0.5,      # 40% → 20% + 20%
    random_state=42
)

In [14]:
print(len(X_train), len(X_val), len(X_test))

27126 9042 9043


## Вопрос 3

Рассчитайте взаимную информацию (mutual_info_classif) между y и другими категориальными переменными в датасете.

Используйте только тренировочный набор.

Округлите результаты до двух знаков после запятой с помощью round(score, 2).

In [15]:
# посмотрим что вообще есть в contact, poutcome
df['contact'].value_counts()

contact
cellular     29285
unknown      13020
telephone     2906
Name: count, dtype: int64

In [16]:
df['poutcome'].value_counts()

poutcome
unknown    36959
failure     4901
other       1840
success     1511
Name: count, dtype: int64

короче скип этот вопрос, не понимаю как делать

In [17]:
mi_scores = mutual_info_classif(
    pd.get_dummies(X_train),
    Y_train,
    random_state=42)

mi_scores

array([0.0139902 , 0.01889414, 0.00902703, 0.07000335, 0.00321284,
       0.02463066, 0.01191604, 0.        , 0.00346503, 0.00153451,
       0.        , 0.00147664, 0.00131441, 0.        , 0.        ,
       0.00202773, 0.00398531, 0.00165127, 0.        , 0.        ,
       0.00588435, 0.00317097, 0.00206853, 0.00060884, 0.00088216,
       0.        , 0.0115358 , 0.01263751, 0.01242816, 0.        ,
       0.01523129, 0.00401889, 0.        , 0.00085848, 0.00166387,
       0.00254586, 0.00111445, 0.        , 0.00375282, 0.00471936,
       0.00072632, 0.00520146, 0.00480916, 0.00022653, 0.00565434,
       0.02743717, 0.0149894 ])

In [18]:
mi = pd.Series(mi_scores, index=pd.get_dummies(X_train).columns)
mi

age                    0.013990
balance                0.018894
day                    0.009027
duration               0.070003
campaign               0.003213
pdays                  0.024631
previous               0.011916
job_admin.             0.000000
job_blue-collar        0.003465
job_entrepreneur       0.001535
job_housemaid          0.000000
job_management         0.001477
job_retired            0.001314
job_self-employed      0.000000
job_services           0.000000
job_student            0.002028
job_technician         0.003985
job_unemployed         0.001651
job_unknown            0.000000
marital_divorced       0.000000
marital_married        0.005884
marital_single         0.003171
education_primary      0.002069
education_secondary    0.000609
education_tertiary     0.000882
education_unknown      0.000000
housing_no             0.011536
housing_yes            0.012638
contact_cellular       0.012428
contact_telephone      0.000000
contact_unknown        0.015231
month_ap

я не знаю что я сделал, списано

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1. One-hot кодирование для всех наборов
X_train_encoded = pd.get_dummies(X_train)
X_val_encoded = pd.get_dummies(X_val)
X_test_encoded = pd.get_dummies(X_test)

# 2. Убедитесь, что все наборы имеют одинаковые столбцы
X_val_encoded = X_val_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# 3. Обучите модель
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_encoded, Y_train)

# 4. Рассчитайте точность на валидационном наборе
y_val_pred = model.predict(X_val_encoded)
accuracy = round(accuracy_score(Y_val, y_val_pred), 2)
print(f"Точность на валидационном наборе: {accuracy}")

Точность на валидационном наборе: 0.9


In [20]:
# Исходная точность (из Q4)
original_accuracy = accuracy

# Словарь для сохранения результатов
feature_importance = {}

# Проходим по каждому признаку
for feature in X_train_encoded.columns:
    # Создаём наборы данных без текущего признака
    X_train_without = X_train_encoded.drop(feature, axis=1)
    X_val_without = X_val_encoded.drop(feature, axis=1)

    # Обучаем модель без этого признака
    model_without = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model_without.fit(X_train_without, Y_train)

    # Рассчитываем точность
    y_val_pred_without = model_without.predict(X_val_without)
    accuracy_without = accuracy_score(Y_val, y_val_pred_without)

    # Вычисляем разницу
    importance = original_accuracy - accuracy_without
    feature_importance[feature] = round(importance, 4)

# Сортируем по важности (убыванию)
sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)

# Выводим результаты
for feature, importance in sorted_features:
    print(f"{feature}: {importance}")



duration: 0.0054
month_jun: 0.0001
job_student: -0.0001
month_may: -0.0001
job_retired: -0.0002
job_services: -0.0002
previous: -0.0004
job_blue-collar: -0.0004
month_mar: -0.0004
month_dec: -0.0005
month_nov: -0.0005
poutcome_success: -0.0005
job_unknown: -0.0006
education_tertiary: -0.0006
contact_telephone: -0.0006
month_jan: -0.0006
pdays: -0.0007
job_admin.: -0.0007
job_housemaid: -0.0007
marital_divorced: -0.0007
marital_single: -0.0007
housing_yes: -0.0007
contact_cellular: -0.0007
month_apr: -0.0007
age: -0.0008
job_self-employed: -0.0008
marital_married: -0.0008
contact_unknown: -0.0008
month_aug: -0.0008
balance: -0.0009
month_feb: -0.0009
month_jul: -0.0009
poutcome_other: -0.0009
job_technician: -0.001
education_primary: -0.001
education_secondary: -0.001
housing_no: -0.001
job_entrepreneur: -0.0011
job_management: -0.0011
job_unemployed: -0.0011
poutcome_failure: -0.0011
day: -0.0012
education_unknown: -0.0012
poutcome_unknown: -0.0012
campaign: -0.0013
month_sep: -0.0015


In [21]:
# Значения параметра C для проверки
C_values = [0.01, 0.1, 1, 10]

# Словарь для сохранения результатов
results = {}

# Обучаем модели с разными C
for C in C_values:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train_encoded, Y_train)

    # Рассчитываем точность
    y_val_pred = model.predict(X_val_encoded)
    accuracy = round(accuracy_score(Y_val, y_val_pred), 3)
    results[C] = accuracy
    print(f"C={C}: {accuracy}")

# Находим лучшее значение C
best_C = max(results, key=results.get)
print(f"\nЛучшее значение C: {best_C} (точность: {results[best_C]})")

C=0.01: 0.899
C=0.1: 0.9
C=1: 0.901
C=10: 0.901

Лучшее значение C: 1 (точность: 0.901)
